In [ ]:
# package imports
from __future__ import annotations

import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, SplineTransformer
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor

from xgboost import XGBRegressor


In [ ]:
# Configuration

DATA_PATH = r"contracts_with_isi_v2_SWEEP_WIDE_WITH_KEYS_PLUS_CPI.csv"

BAT_RATES_PATH = r"batting_rates_by_season.csv"
PIT_RATES_PATH = r"pitching_rates_by_season.csv"

# NEW
DEF_STATS_PATH = r"defensive_stats.csv"
STATCAST_PIT_PATH = r"statcast_pitching_2015_2025.csv"

# Target
REG_TARGET = "guarantee_real_per_year_2025"

# Time split
TRAIN_YEARS = {2020, 2021, 2022, 2023}
TEST_YEARS  = {2024, 2025}

# Window settings for panels
PRE_YEARS = 3

# Exports
OUT_REG_RESULTS = r"model_comparison_regression_BASELINE_new.csv"


# Base features
BASE_NUMERIC = [
    "age_at_signing",
    "years_int",
    "opt_out_flag",
    "year",
    "is_pitcher_flag",
]

BASE_CATEGORICAL = [
    "position",
    "qualifying_offer",
]

In [ ]:
# Helper functions

# drop dups, preserves order
def unique_list(seq):
    return list(dict.fromkeys(seq))

# drop dup col names
def dedupe_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.loc[:, ~df.columns.duplicated()].copy()

PITCHER_PREFIXES = ("P", "SP", "RP", "RHP", "LHP")

def is_pitcher(pos) -> int:
    if pd.isna(pos):
        return 0
    s = str(pos).strip().upper()
    # handles pitcher positional variations
    return int(s.startswith(PITCHER_PREFIXES) or ("RHP" in s) or ("LHP" in s))

def time_split(df: pd.DataFrame, year_col: str = "year") -> Tuple[pd.DataFrame, pd.DataFrame]:
    y = pd.to_numeric(df[year_col], errors="coerce").astype("Int64")
    train = df[y.isin(TRAIN_YEARS)].copy()
    test  = df[y.isin(TEST_YEARS)].copy()
    return train, test

def regression_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)

    # Data integrity check
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    if mask.sum() == 0:
        raise ValueError("No y_true/y_pred pairs available after filtering.")

    y_true = y_true[mask]
    y_pred = y_pred[mask]

    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae = float(mean_absolute_error(y_true, y_pred))
    r2 = float(r2_score(y_true, y_pred))
    return {"RMSE": rmse, "MAE": mae, "R2": r2, "n_eval": int(mask.sum())}

def _safe_numeric(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    out = df.copy()
    for c in cols:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

def _weighted_mean(series: pd.Series, weights: pd.Series) -> float:
    s = pd.to_numeric(series, errors="coerce")
    w = pd.to_numeric(weights, errors="coerce").fillna(0.0)
    mask = np.isfinite(s) & np.isfinite(w) & (w > 0)
    if mask.sum() == 0:
        s2 = s[np.isfinite(s)]
        return float(s2.mean()) if len(s2) else np.nan
    return float(np.average(s[mask], weights=w[mask]))

def add_pre_rate_features(
    contracts: pd.DataFrame,
    season_rates: pd.DataFrame,
    *,
    rate_cols: List[str],
    weight_col: str | None,
    prefix: str,
    pre_years: int = 3,
) -> pd.DataFrame:
    """
    Pre-window aggregate computations for seasons in window
    """
    dfc = contracts.copy()
    dfc["key_fangraphs"] = pd.to_numeric(dfc["key_fangraphs"], errors="coerce")
    dfc["year"] = pd.to_numeric(dfc["year"], errors="coerce")
    dfc["_row_id"] = np.arange(len(dfc), dtype=int)

    dfs = season_rates.copy()
    dfs["playerId"] = pd.to_numeric(dfs["playerId"], errors="coerce")
    dfs["Season"] = pd.to_numeric(dfs["Season"], errors="coerce")

    dfs = _safe_numeric(dfs, rate_cols + ([weight_col] if weight_col else []))

    m = dfc[["_row_id", "key_fangraphs", "year"]].merge(
        dfs,
        left_on="key_fangraphs",
        right_on="playerId",
        how="left",
    )

    m["lb_start"] = m["year"] - pre_years
    m["lb_end"] = m["year"] - 1
    m = m[m["Season"].between(m["lb_start"], m["lb_end"], inclusive="both")].copy()

    out = dfc.copy()

    cov = m.groupby("_row_id")["Season"].nunique().rename(f"{prefix}_pre_seasons")
    out = out.merge(cov, left_on="_row_id", right_index=True, how="left")
    out[f"{prefix}_pre_seasons"] = out[f"{prefix}_pre_seasons"].fillna(0).astype(int)

    if weight_col and weight_col in m.columns:
        rel_sum = m.groupby("_row_id")[weight_col].sum(min_count=1).rename(f"{prefix}_pre_reliability_sum")
        out = out.merge(rel_sum, left_on="_row_id", right_index=True, how="left")
        out[f"{prefix}_pre_reliability_sum"] = pd.to_numeric(out[f"{prefix}_pre_reliability_sum"], errors="coerce").fillna(0.0)

    for rc in rate_cols:
        feat_name = f"{prefix}_pre_{rc}"
        if rc not in m.columns:
            out[feat_name] = np.nan
            continue
        if weight_col and weight_col in m.columns:
            agg = m.groupby("_row_id").apply(lambda g: _weighted_mean(g[rc], g[weight_col])).rename(feat_name)
        else:
            agg = m.groupby("_row_id")[rc].mean().rename(feat_name)
        out = out.merge(agg, left_on="_row_id", right_index=True, how="left")

    out[f"has_{prefix}_pre"] = (out[f"{prefix}_pre_seasons"] > 0).astype(int)
    out = out.drop(columns=["_row_id"])
    return out


In [ ]:
# apply pre panel features

def add_pre_panel_features(
    contracts: pd.DataFrame,
    panel: pd.DataFrame,
    *,
    contract_key_col: str,
    panel_key_col: str,
    contract_year_col: str,
    panel_year_col: str,
    feature_cols: List[str],
    weight_col: str | None,
    prefix: str,
    pre_years: int = 3,
) -> pd.DataFrame:
    """ Generic season panel aggregator (only used for defense and statcast files) """
    # Data integrity
    # Missing value check
    missing_contract = [c for c in [contract_key_col, contract_year_col] if c not in contracts.columns]
    missing_panel = [c for c in [panel_key_col, panel_year_col] if c not in panel.columns]
    if missing_contract:
        raise KeyError(f"[add_pre_panel_features] contracts missing: {missing_contract}")
    if missing_panel:
        raise KeyError(f"[add_pre_panel_features] panel missing: {missing_panel}")

    out = contracts.copy()
    out["_row_id"] = np.arange(len(out), dtype=int)

    # temp cols
    dfc = out[["_row_id", contract_key_col, contract_year_col]].copy()
    dfc["_contract_key"] = pd.to_numeric(dfc[contract_key_col], errors="coerce")
    dfc["_contract_year"] = pd.to_numeric(dfc[contract_year_col], errors="coerce")

    # returns necessary cols only
    keep_feats = [c for c in feature_cols if c in panel.columns]
    keep_cols = [panel_key_col, panel_year_col] + keep_feats
    if weight_col and weight_col in panel.columns and weight_col not in keep_cols:
        keep_cols.append(weight_col)

    p = panel[keep_cols].copy()
    p["_panel_key"] = pd.to_numeric(p[panel_key_col], errors="coerce")
    p["_panel_year"] = pd.to_numeric(p[panel_year_col], errors="coerce")

    for c in keep_feats:
        p[c] = pd.to_numeric(p[c], errors="coerce")
    if weight_col and weight_col in p.columns:
        p[weight_col] = pd.to_numeric(p[weight_col], errors="coerce")

    merge_cols = ["_panel_key", "_panel_year"] + keep_feats + ([weight_col] if (weight_col and weight_col in p.columns) else [])
    m = dfc[["_row_id", "_contract_key", "_contract_year"]].merge(
        p[merge_cols],
        left_on="_contract_key",
        right_on="_panel_key",
        how="left",
    )

    # Apply lookback filter
    m["lb_start"] = m["_contract_year"] - pre_years
    m["lb_end"] = m["_contract_year"] - 1
    m = m[m["_panel_year"].between(m["lb_start"], m["lb_end"], inclusive="both")].copy()

    cov = m.groupby("_row_id")["_panel_year"].nunique().rename(f"{prefix}_pre_seasons")
    out = out.merge(cov, on="_row_id", how="left")
    out[f"{prefix}_pre_seasons"] = out[f"{prefix}_pre_seasons"].fillna(0).astype(int)

    # Weighted sums
    if weight_col and weight_col in m.columns:
        wsum = m.groupby("_row_id")[weight_col].sum(min_count=1).rename(f"{prefix}_pre_weight_sum")
        out = out.merge(wsum, on="_row_id", how="left")
        out[f"{prefix}_pre_weight_sum"] = pd.to_numeric(out[f"{prefix}_pre_weight_sum"], errors="coerce").fillna(0.0)

    # Aggregate features where necessary
    if weight_col and weight_col in m.columns:
        w = pd.to_numeric(m[weight_col], errors="coerce").fillna(0.0)

        for fc in keep_feats:
            feat_name = f"{prefix}_pre_{fc}"
            x = pd.to_numeric(m[fc], errors="coerce")
            num = (x * w).groupby(m["_row_id"]).sum(min_count=1)
            den = w.groupby(m["_row_id"]).sum(min_count=1)
            wmean = (num / den).replace([np.inf, -np.inf], np.nan).rename(feat_name)
            out = out.merge(wmean, on="_row_id", how="left")
    else:
        for fc in keep_feats:
            feat_name = f"{prefix}_pre_{fc}"
            mean = m.groupby("_row_id")[fc].mean().rename(feat_name)
            out = out.merge(mean, on="_row_id", how="left")

    out[f"has_{prefix}_pre"] = (out[f"{prefix}_pre_seasons"] > 0).astype(int)

    out = out.drop(columns=["_row_id"])
    return out



In [ ]:
# Data Preprocessing

def make_preprocessor(
    numeric_features: List[str],
    categorical_features: List[str],
    use_splines: bool
) -> ColumnTransformer:
    num_steps = [("imputer", SimpleImputer(strategy="median"))]

    if use_splines:
        num_steps += [
            ("splines", SplineTransformer(n_knots=6, degree=3, include_bias=False)),
            ("scaler", StandardScaler(with_mean=False)),
        ]
    else:
        num_steps += [("scaler", StandardScaler())]

    num_pipe = Pipeline(steps=num_steps)
    cat_pipe = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    return ColumnTransformer(
        transformers=[
            ("num", num_pipe, numeric_features),
            ("cat", cat_pipe, categorical_features),
        ],
        remainder="drop",
        sparse_threshold=0.3,
    )

In [ ]:
# Model Creation
# Using LR, EN, RFG, GAMSL, GAMSEN, XGB Sweep
def get_regression_models(random_state: int = 42) -> Dict[str, object]:
    models = {
        "LinearRegression": LinearRegression(),
        "ElasticNet": ElasticNet(alpha=0.002, l1_ratio=0.35, random_state=random_state, max_iter=20000),
        "RandomForest": RandomForestRegressor(
            n_estimators=800,
            max_depth=None,
            min_samples_leaf=2,
            random_state=random_state,
            n_jobs=-1,
        ),
        "GAM_Splines_Linear": LinearRegression(),
        "GAM_Splines_ElasticNet": ElasticNet(alpha=0.0015, l1_ratio=0.2, random_state=random_state, max_iter=20000),
    }

    # XGB sweep config
    xgb_grid = [
        {"n_estimators": 600,  "learning_rate": 0.05, "max_depth": 3, "subsample": 0.85, "colsample_bytree": 0.85, "reg_lambda": 1.0},
        {"n_estimators": 900,  "learning_rate": 0.04, "max_depth": 3, "subsample": 0.85, "colsample_bytree": 0.85, "reg_lambda": 1.0},
        {"n_estimators": 1200, "learning_rate": 0.03, "max_depth": 4, "subsample": 0.85, "colsample_bytree": 0.85, "reg_lambda": 1.0},
        {"n_estimators": 1400, "learning_rate": 0.025,"max_depth": 4, "subsample": 0.80, "colsample_bytree": 0.85, "reg_lambda": 1.5},
        {"n_estimators": 1600, "learning_rate": 0.02, "max_depth": 5, "subsample": 0.80, "colsample_bytree": 0.80, "reg_lambda": 2.0},
        {"n_estimators": 1000, "learning_rate": 0.001,"max_depth": 3, "subsample": 0.85, "colsample_bytree": 0.85, "reg_lambda": 1.0},
        {"n_estimators": 1000, "learning_rate": 0.005,"max_depth": 3, "subsample": 0.85, "colsample_bytree": 0.85, "reg_lambda": 1.0},
        {"n_estimators": 2500, "learning_rate": 0.001,"max_depth": 4, "subsample": 0.85, "colsample_bytree": 0.85, "reg_lambda": 1.0},
        {"n_estimators": 2500, "learning_rate": 0.005,"max_depth": 4, "subsample": 0.80, "colsample_bytree": 0.85, "reg_lambda": 1.5},
        {"n_estimators": 5000, "learning_rate": 0.001,"max_depth": 5, "subsample": 0.80, "colsample_bytree": 0.80, "reg_lambda": 2.0},
        {"n_estimators": 5000, "learning_rate": 0.005,"max_depth": 5, "subsample": 0.80, "colsample_bytree": 0.80, "reg_lambda": 2.0},
    ]

    # XGB Setup (applied once, rather than for each sweep iteration)
    xgb_defaults = dict(
        random_state=random_state,
        objective="reg:squarederror",
        n_jobs=-1,
        reg_alpha=0.0,
        min_child_weight=5,
    )

    # assigns a name for each XGB sweep model
    # ensures no overwriting of previous iterations
    for i, params in enumerate(xgb_grid, start=1):
        name = f"XGB_{i:02d}_ne{params['n_estimators']}_lr{params['learning_rate']}_md{params['max_depth']}"
        models[name] = XGBRegressor(**xgb_defaults, **params)

    return models

# Model training
def fit_predict_regression(
    model_name: str, model, X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame,
    *, numeric_features: List[str], categorical_features: List[str],
    ) -> Tuple[np.ndarray, object]:

    use_splines = model_name.startswith("GAM_Splines_")
    pre = make_preprocessor(numeric_features, categorical_features, use_splines=use_splines)

    reg = Pipeline(steps=[("pre", pre), ("model", model)])
    ttr = TransformedTargetRegressor(regressor=reg, func=np.log1p, inverse_func=np.expm1)

    ttr.fit(X_train, y_train)
    preds = ttr.predict(X_test)
    return preds, ttr


In [ ]:
# Main runs baseline regression models that are set up above

if __name__ == "__main__":

    # data loader
    df = pd.read_csv(DATA_PATH)

    bat_rates = pd.read_csv(BAT_RATES_PATH)
    pit_rates = pd.read_csv(PIT_RATES_PATH)
    def_stats = pd.read_csv(DEF_STATS_PATH)
    sc_pit    = pd.read_csv(STATCAST_PIT_PATH)

    # Data integrity
    df = dedupe_columns(df)

    df["term_start_year"] = pd.to_numeric(df.get("term_start_year"), errors="coerce")
    df = df.dropna(subset=["term_start_year"]).copy()
    df["year"] = pd.to_numeric(df["term_start_year"], errors="coerce")

    # Restrict contract years are 2020-2025
    df = df[(df["year"] >= 2020) & (df["year"] <= 2025)].copy()

    # Restricts contract terms to 1-5 years
    MAX_YEARS = 5
    df["years_int"] = pd.to_numeric(df["years_int"], errors="coerce")

    before = len(df)
    df = df[df["years_int"].notna() & (df["years_int"] <= MAX_YEARS)].copy()
    after = len(df)

    print(f"[FILTER] Dropped {before - after} contracts with years_int > {MAX_YEARS} (or missing)")

    # coerce keys
    for c in ["key_fangraphs", "key_mlbam"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # Assign pitcher flag
    if "position" in df.columns:
        df["is_pitcher_flag"] = df["position"].map(is_pitcher).astype(int)
    else:
        df["is_pitcher_flag"] = 0

    # Data integrity
    # Confirms necessary cols are present
    print("\n--- DEBUG: before panel merges ---")
    print("df has year?", "year" in df.columns)
    print("df year dtype:", df["year"].dtype)
    print("df key_mlbam dtype:", df["key_mlbam"].dtype if "key_mlbam" in df.columns else None)
    print("df key_fangraphs dtype:", df["key_fangraphs"].dtype if "key_fangraphs" in df.columns else None)
    print("sc_pit has year?", "year" in sc_pit.columns)
    print("def_stats has year?", "year" in def_stats.columns)


    BAT_EXCLUDE = {"playerId", "Season", "Name", "Tm", "PA"}
    PIT_EXCLUDE = {"playerId", "Season", "Name", "Tm", "IP", "TBF"}

    bat_rate_cols = [
        c for c in bat_rates.columns
        if c not in BAT_EXCLUDE
        and c != "bat_rate_reliability"
        and not c.endswith("_dup")
    ]
    pit_rate_cols = [
        c for c in pit_rates.columns
        if c not in PIT_EXCLUDE
        and c != "pit_rate_reliability"
        and not c.endswith("_dup")
    ]

    # Data integrity
    bat_weight_col = "bat_rate_reliability" if "bat_rate_reliability" in bat_rates.columns else None
    pit_weight_col = "pit_rate_reliability" if "pit_rate_reliability" in pit_rates.columns else None

    df = add_pre_rate_features(
        df,
        bat_rates,
        rate_cols=bat_rate_cols,
        weight_col=bat_weight_col,
        prefix="bat",
        pre_years=PRE_YEARS,
    )

    df = add_pre_rate_features(
        df,
        pit_rates,
        rate_cols=pit_rate_cols,
        weight_col=pit_weight_col,
        prefix="pit",
        pre_years=PRE_YEARS,
    )


    # Add defensive features

    def_feature_cols = [c for c in ["defensive_runs_saved", "fielding_percentage", "Errors"] if c in def_stats.columns]

    df = add_pre_panel_features(
        contracts=df,
        panel=def_stats,
        contract_key_col="key_mlbam",
        panel_key_col="MLBAMID",
        contract_year_col="year",
        panel_year_col="year",
        feature_cols=def_feature_cols,
        weight_col="Innings_played" if "Innings_played" in def_stats.columns else None,
        prefix="def",
        pre_years=PRE_YEARS,
    )

    # Add statcast pitching features
    sc_feature_cols = [
        c for c in [
            "fastball_avg_speed",
            "whiff_percent",
            "hard_hit_percent",
            "barrel_batted_rate",
            "exit_velocity_avg",
            "swing_percent",
        ]
        if c in sc_pit.columns
    ]

    df = add_pre_panel_features(
        contracts=df,
        panel=sc_pit,
        contract_key_col="key_mlbam",
        panel_key_col="player_id",
        contract_year_col="year",
        panel_year_col="year",
        feature_cols=sc_feature_cols,
        weight_col="pa" if "pa" in sc_pit.columns else None,
        prefix="scpit",
        pre_years=PRE_YEARS,
    )

    df = dedupe_columns(df)

    # Data Integrity
    for col in ["has_bat_pre", "has_pit_pre", "has_def_pre", "has_scpit_pre"]:
        if col in df.columns:
            print(f"{col}: {df[col].mean():.3f} coverage ({int(df[col].sum())}/{len(df)})")

    # Audit
    DUP_KEY = [
        "player_name",
        "term_start_year",
        "new_club",
        "years_int",
        REG_TARGET,
    ]
    DUP_KEY = [c for c in DUP_KEY if c in df.columns]
    dups = df[df.duplicated(DUP_KEY, keep=False)].sort_values(DUP_KEY) if len(DUP_KEY) else pd.DataFrame()

    if len(dups):
        dups.to_csv(r"dup_contract_audit_BASELINE.csv", index=False)
        print(f"Saved dup audit: {len(dups)} rows")

    if len(DUP_KEY):
        df = df.drop_duplicates(DUP_KEY, keep="first").copy()

    print("\nDuplicate audit:")
    print("Key columns used:", DUP_KEY)
    print("Total rows:", len(df))
    print("Duplicate rows (including both originals and copies):", len(dups))


    # Runs Regresion Model
    df_reg = df.dropna(subset=[REG_TARGET]).copy()
    df_reg = dedupe_columns(df_reg)

    # Final Integrity check
    # ensures target row is cleaned and usable
    df_reg[REG_TARGET] = (
        df_reg[REG_TARGET]
        .astype(str)
        .str.replace(r"[\$,]", "", regex=True)
        .str.strip()
    )
    df_reg[REG_TARGET] = pd.to_numeric(df_reg[REG_TARGET], errors="coerce")
    df_reg = df_reg[np.isfinite(df_reg[REG_TARGET])].copy()
    df_reg = df_reg[df_reg[REG_TARGET] >= 0].copy()

    train_reg_full, test_reg_full = time_split(df_reg, year_col="year")

    cov_suffixes = ("_pre_seasons", "_pre_reliability_sum", "_pre_weight_sum")
    generated_cov_feats = [
        c for c in df_reg.columns
        if c in {"has_bat_pre", "has_pit_pre", "has_def_pre", "has_scpit_pre"}
        or c.endswith(cov_suffixes)
    ]

    PREFIXES = ("bat_pre_", "pit_pre_", "def_pre_", "scpit_pre_")
    generated_rate_feats = [
        c for c in df_reg.columns
        if c.startswith(PREFIXES) and c not in generated_cov_feats
    ]

    numeric_features = unique_list([
        c for c in (BASE_NUMERIC + generated_rate_feats + generated_cov_feats)
        if c in df_reg.columns
    ])
    categorical_features = unique_list([
        c for c in (BASE_CATEGORICAL)
        if c in df_reg.columns
    ])

    # standard with no percentile cutoff
    def run_variant(train_reg: pd.DataFrame, test_reg: pd.DataFrame, *, variant: str, cut_pct: float | None, cut_value: float | None):
        X_train = train_reg[numeric_features + categorical_features]
        y_train = pd.to_numeric(train_reg[REG_TARGET], errors="coerce")
        X_test  = test_reg[numeric_features + categorical_features]
        y_test  = pd.to_numeric(test_reg[REG_TARGET], errors="coerce")

        rows = []
        for model_name, model in get_regression_models().items():
            preds, _ = fit_predict_regression(
                model_name, model, X_train, y_train, X_test,
                numeric_features=numeric_features,
                categorical_features=categorical_features
            )
            mets = regression_metrics(y_test.values, preds)

            rows.append({
                "run": "BASELINE_ONLY",
                "variant": variant,                 
                "cut_pct": cut_pct,             
                "cut_value": cut_value,
                "model": model_name,
                "n_train": len(train_reg),
                "n_test": len(test_reg),
                "n_features_numeric": len(numeric_features),
                "n_features_categorical": len(categorical_features),
                **mets,
            })
        return rows

    reg_results_rows = []

    # Unfiltered variant
    reg_results_rows += run_variant(
        train_reg_full, test_reg_full,
        variant="UNFILTERED",
        cut_pct=None,
        cut_value=None,
    )

    # Variant 5% Cutoff
    REMOVE_TOP_PCTL = 0.95
    aav_cutoff = train_reg_full[REG_TARGET].quantile(REMOVE_TOP_PCTL)

    train_reg_cut = train_reg_full[train_reg_full[REG_TARGET] <= aav_cutoff].copy()
    test_reg_cut  = test_reg_full[test_reg_full[REG_TARGET] <= aav_cutoff].copy()

    print(f"[TOP5% FILTER] Training Contract Cutoff ({REMOVE_TOP_PCTL:.0%}): {aav_cutoff:,.0f}")
    print(f"[TOP5% FILTER] n_train={len(train_reg_cut)} | n_test={len(test_reg_cut)}")

    reg_results_rows += run_variant(
        train_reg_cut, test_reg_cut,
        variant="TOP5_REMOVED",
        cut_pct=REMOVE_TOP_PCTL,
        cut_value=float(aav_cutoff),
    )

    reg_results = pd.DataFrame(reg_results_rows).sort_values(["variant", "MAE"])
    reg_results.to_csv(OUT_REG_RESULTS, index=False)

    print("\nSaved regression comparison:", OUT_REG_RESULTS)

    # prints top 5 models for each variant
    for v in reg_results["variant"].unique():
        print(f"\nTop results for {v} (sorted by MAE):")
        sub = reg_results[reg_results["variant"] == v].sort_values("MAE").head(10)
        print(sub[["model", "MAE", "RMSE", "R2", "n_train", "n_test", "cut_value"]].round(4).to_string(index=False))
